###

# GOR Method Implementation

In [1]:
import pandas as pd
import numpy as np
import math
import os

1. Build a GOR propensity matrix from the files in the propensity folder (for H,E and '-' categories)
2. Create inference method to predict sites in protein sequence via GOR method
3. Compare against secondary structure sequences from PDB

## Create GOR Propensity Matrices

### Create Frequency Tables

Output is three tables: one each for H, E, and '-'
Each table has a row for each position relative to the focal residue and a column for each AA (The X column is dropped).
I'm open to some swaps on that output, but I suspect it will make life a little easier

In [80]:
#Rows are in same order as the table in original GOR paper
AA_LETTERS = ['G', 'A', 'V', 'L', 'I', 'S', 'T', 'D', 'E', 'N', 'Q', 'K', 'H', 'R', 'F', 'Y', 'W', 'C', 'M', 'P', 'X']
STRUCTURE_KEYS = ['H', 'E', '-']
WINDOW_RADIUS = 8

# The X is a way to handle edges via padding, and is zeroed before calculating frequencies

In [14]:
def init_count_table():
    return pd.DataFrame(0,index=np.arange(17),columns=AA_LETTERS)

def init_class_counts():
    return {'H': init_count_table(), 'E': init_count_table(), '-': init_count_table()}

In [15]:
count_tables = init_class_counts()

In [ ]:
def count_seq_file(seqfile):
    seq_class_counts = init_class_counts()
    with open(seqfile, 'r') as f:
        lines = f.readlines()
        structure = lines[0].strip()
        sequence = lines[1].strip()
        padding = 'X' * 8
        padded_seq = padding + sequence + padding
    for i in range(0,len(structure)):
        category = structure[i]
        window_seqs = padded_seq[i:i+17]
        for j in range(0,len(window_seqs)):
            seq_class_counts[category].loc[j, window_seqs[j]] += 1
    return seq_class_counts


In [17]:
directory = r"train/"
for filename in os.listdir(directory):
    seq_class_counts = count_seq_file(os.path.join(directory,filename))
    for key in seq_class_counts.keys():
        count_tables[key] += seq_class_counts[key]

In [18]:
trimmed_count_tables = count_tables.copy()
for key in count_tables.keys():
    trimmed_count_tables[key]["X"] = 0

### Create Frequency Tables from Count Tables

In [48]:
cond_freq_tables = {key: trimmed_count_tables[key].div(trimmed_count_tables[key].sum(axis=1),axis=0) for key in STRUCTURE_KEYS}

In [20]:
structure_totals = {key: trimmed_count_tables[key].loc[8].sum() for key in STRUCTURE_KEYS}   
n_residues = sum(structure_totals.values()) 
structure_tot_freqs = {key: structure_totals[key]/n_residues for key in STRUCTURE_KEYS}
structure_tot_freqs

{'H': 0.3549997224198929, 'E': 0.22097899980317046, '-': 0.4240212777769366}

In [30]:
total_aa_counts = trimmed_count_tables["H"] + trimmed_count_tables["E"] + trimmed_count_tables["-"]
overall_aa_freqs = total_aa_counts.div(total_aa_counts.sum(axis=1),axis=0)

### Create Information Tables From Frequency Tables

In [ ]:
info_tables = {key: np.log2(cond_freq_tables[key]/overall_aa_freqs) for key in STRUCTURE_KEYS}
## Knuth forgive me for the kludge I am about to perform
for key in info_tables.keys():
    info_tables[key]["X"] = 0

## Inference Method

In [90]:
def predict_secondary_structure(sequence, info_tables):
    WINDOW_RADIUS = 8
    sequence = sequence.upper()
    length = len(sequence)
    pred_ss = []

    for pos in range(length):
        scores = {'H': 0.0, 'E': 0.0, '-': 0.0}
        # Slide window centered on residue pos
        for offset in range(-WINDOW_RADIUS, WINDOW_RADIUS+1):
            j = pos + offset

            # Skip positions that go outside sequence
            if j < 0 or j >= length:
                continue

            aa = sequence[j]
            if aa not in AA_LETTERS:
                continue

            # Add contribution from info table for each structure type
            for key in STRUCTURE_KEYS:
                scores[key] += info_tables[key].loc[offset + WINDOW_RADIUS, aa]

        # Choose structure with highest total score
        best_state = max(scores, key=scores.get)
        pred_ss.append(best_state)

    return ''.join(pred_ss)

## Evaluate Against File

In [94]:
def hamming_distance(seq1, seq2):
    return sum(s1 != s2 for s1, s2 in zip(seq1, seq2))

def evaluate_file(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        struct_target = lines[0].strip()
        seq_target = lines[1].strip()
    struct_pred = predict_secondary_structure(seq_target, info_tables)
    distance = hamming_distance(struct_target, struct_pred)
    acc = distance / len(struct_target)
    return [acc, struct_pred, struct_target]



In [100]:
test_file = "train/23.dssp" #Fairly long sequenve with stretches of all three structure classes
test_acc, struct_pred, struct_target  = evaluate_file(test_file)
print(f"pred: {struct_pred}")
print(f"trgt: {struct_target}")
print(f"accuracy: {round(test_acc * 100,2)}%")

pred: EEEEEEEEE----HHEEEEEE------EEEEEE-----EEEEEE-------EEEEEE----------E-EEEEE-----E----------EE--E-----HHHHHHHHHHHHHHHHHHHEEE------EEEEEEE---------EEEE--EEEEEE------EEEE----HHEEEE------EEEEEE----EEEE---HH-EE--HHHH-EEEEEE----H----HHHEEEE------HHHHHHHHHH------H--EEEEEEE----HHHHHHEE-------------EEE----E
trgt: -EEEEEEEE---HHHHHHHH---------HHHHHHHHHHHHHHH-------EEEEEE----------EEE-EEEE---EE-E-E----E-----EE--HHHHHHHHHHHHH-----EEE----E-HHHHHHHHHHH--------EEEEEEE---------HHHHHHHHHHHHHHHH------EEEEEEE-------------E--HHHHHHHHHHHHH-HHHH----HHHHHHHH---------HHHHHH-----EEEEEEEEEEEE--EEEEEEEEEE-----------EEEE----
accuracy: 43.29%
